In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import matplotlib.pyplot as plt
from math import *
import time

In [ ]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [ ]:
# activation function
def activation(x):
    return x * torch.sigmoid(x) 

In [ ]:
# build ResNet with one blocks
class Net(nn.Module):
    def __init__(self,input_size,width):
        super(Net,self).__init__()
        self.layer_in = nn.Linear(input_size,width)
        self.layer_1 = nn.Linear(width,width)
        self.layer_2 = nn.Linear(width,width)
        self.layer_out = nn.Linear(width,1)
    def forward(self,x):
        output = self.layer_in(x)
        output = output + activation(self.layer_2(activation(self.layer_1(output)))) # residual block 1
        output = self.layer_out(output)
        return output

In [ ]:
input_size = 1
width = 5
net = Net(input_size,width)

In [ ]:
def model(x):
    return x * (x - 1.0) * net(x)

In [ ]:
# exact solution
def u_ex(x):  
    return torch.sin(pi*x)

In [ ]:
# f(x)
def f(x):
    return pi**2 * torch.sin(pi*x)

In [ ]:
grid_num = 200
x = torch.zeros(grid_num + 1, input_size)
for index in range(grid_num + 1):
    x[index] = index * 1 / grid_num

In [ ]:
# loss function to DGM by auto differential
def loss_function(x):
    h = 1 / grid_num
    sum_0 = 0.0
    sum_1 = 0.0
    sum_2 = 0.0
    sum_a = 0.0
    sum_b = 0.0
    for index in range(grid_num):
        x_temp = x[index] + h / 2 
        x_temp.requires_grad = True
#         grad_x_temp = torch.autograd.grad(model(x_temp), x_temp, create_graph = True)
        grad_x_temp = torch.autograd.grad(outputs = model(x_temp), inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
        grad_grad_x_temp = torch.autograd.grad(outputs = grad_x_temp[0], inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
        sum_1 += ((grad_grad_x_temp[0])[0] + f(x_temp)[0])**2
    
    for index in range(1, grid_num):
        x_temp = x[index]
        x_temp.requires_grad = True
#         grad_x_temp = torch.autograd.grad(model(x_temp), x_temp, create_graph = True)
        grad_x_temp = torch.autograd.grad(outputs = model(x_temp), inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
        grad_grad_x_temp = torch.autograd.grad(outputs = grad_x_temp[0], inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
        sum_2 += ((grad_grad_x_temp[0])[0] + f(x_temp)[0])**2
    
    x_temp = x[0]
    x_temp.requires_grad = True
#     grad_x_temp = torch.autograd.grad(model(x_temp), x_temp, create_graph = True)
    grad_x_temp = torch.autograd.grad(outputs = model(x_temp), inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
    grad_grad_x_temp = torch.autograd.grad(outputs = grad_x_temp[0], inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
    sum_a = ((grad_grad_x_temp[0])[0] + f(x_temp)[0])**2
    
    x_temp = x[grid_num]
    x_temp.requires_grad = True
#     grad_x_temp = torch.autograd.grad(model(x_temp), x_temp, create_graph = True)
    grad_x_temp = torch.autograd.grad(outputs = model(x_temp), inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
    grad_grad_x_temp = torch.autograd.grad(outputs = grad_x_temp[0], inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
    sum_b = ((grad_grad_x_temp[0])[0] + f(x_temp)[0])**2
    
    sum_0 = h / 6 * (sum_a + 4 * sum_1 + 2 * sum_2 + sum_b)
    return sum_0

In [ ]:
def error_function(x):
    error = 0.0
    for index in range(len(x)):
        x_temp = x[index]
        error += (model(x_temp)[0] - u_ex(x_temp)[0])**2
    return error / len(x)

In [ ]:
# set optimizer and learning rate decay
optimizer = optim.Adam(net.parameters())
scheduler = lr_scheduler.StepLR(optimizer, 250, 0.8) # every 250 epoch, learning rate * 0.5

In [ ]:
epoch = 5000
loss_record = np.zeros(epoch)
error_record = np.zeros(epoch)
time_start = time.time()
for i in range(epoch):
    optimizer.zero_grad()
    loss = loss_function(x)
    loss_record[i] = float(loss)
    error = error_function(x)
    error_record[i] = float(error)
    np.save("unit_DGM_loss_1d_5.npy", loss_record)
    np.save("unit_DGM_error_1d_5.npy", error_record)
    if i % 1 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", loss.detach())
        print("current error is: ", error.detach())
        
    loss.backward()
    optimizer.step() 
    scheduler.step()
    torch.cuda.empty_cache() # clear memory
    
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')